In [1]:
import pandas  as pd
import numpy as np

In [2]:
store = pd.read_csv('spb/data_2/store_cleaned.csv')

In [7]:
store.head()

,month_,prod_norm_name,prod_form_norm_name,prod_d_norm_name,prod_pack_1_2,prod_pack_1_size,ЖНВЛП,Ковид,origin,code,name,Count_3,is_sales,year_
0,8,МНН №1,МАЗЬ НАЗАЛЬНАЯ,0.25 %,1,10.0,0,0,1,33,Владимирская область,2767,False,2021
1,8,МНН №9,КАПЛИ ГЛАЗНЫЕ,20 %,1,10.0,0,0,1,72,Тюменская область,1981,False,2021
2,8,МНН №9,КАПЛИ ГЛАЗНЫЕ,20 %,1,10.0,0,0,1,75,Забайкальский край,1106,False,2021
3,8,МНН №14,КАПЛИ ГЛАЗНЫЕ,0.007 %,1,5.0,0,0,1,63,Самарская область,156,False,2021
4,8,МНН №32,ГЕЛЬ ДЛЯ НАРУЖНОГО ПРИМЕНЕНИЯ,0.1 %,1,30.0,0,0,2,54,Новосибирская область,588,False,2021


In [8]:
store = store[['name', 'Ковид', 'Count_3', 'month_', 'prod_norm_name']]

In [9]:
store

,name,Ковид,Count_3,month_,prod_norm_name
0,Владимирская область,0,2767,8,МНН №1
1,Тюменская область,0,1981,8,МНН №9
2,Забайкальский край,0,1106,8,МНН №9
3,Самарская область,0,156,8,МНН №14
4,Новосибирская область,0,588,8,МНН №32
...,...,...,...,...,...
4217732,Оренбургская область,0,1,1,МНН №1189
4217733,Карачаево-Черкесская республика,0,1,1,МНН №334
4217734,Брянская область,0,1,1,МНН №299
4217735,Республика Тыва,0,1,1,МНН №408


In [139]:
covid = pd.read_csv('spb/covid_full_stat_v2.csv', parse_dates=True, index_col='Дата', dayfirst=True)
covid = covid.sort_index()
covid = covid['2021':'2021-10']
covid['month_'] = covid.index.month
covid = covid[['month_', 'name_store', 'Заражений за день']]
covid

,month_,name_store,Заражений за день
Дата,,,
2021-01-01,1,Республика Башкортостан,170
2021-01-01,1,Белгородская область,190
2021-01-01,1,Липецкая область,163
2021-01-01,1,Псковская область,303
2021-01-01,1,Чукотский автономный округ,1
...,...,...,...
2021-10-31,10,Московская область,2737
2021-10-31,10,Курганская область,206
2021-10-31,10,Еврейская автономная область,45


In [19]:
months = np.array(range(2, 11))
months

array([ 2,  3,  4,  5,  6,  7,  8,  9, 10])

In [123]:
len(set(covid['name_store']) & set(store['name']))

77

In [124]:
len(covid['name_store'].unique())

85

In [126]:
len(store['name'].unique())

87

In [149]:
%%time

train_mtx = np.array([]).reshape(0, 5)
months = np.array(range(3, 11))

covid['name_store'] = covid['name_store'].str.strip()

for region in store['name'].unique():
    region = region.strip()
    if region == 'РФ':
        continue
        
    df = store.query('name == @region and Ковид')
    region_covid = covid.query('name_store == @region') \
                        .resample('MS')['Заражений за день'] \
                        .sum().pct_change().shift().values
    
#     if len(region_covid) > 0:
    region_values = np.array([region] * len(months))

    for mnn in df['prod_norm_name'].unique():
        df_mnn = df.query('prod_norm_name == @mnn')
        values_mnn = df_mnn[['month_', 'Count_3']].groupby('month_') \
                                     .sum().diff().pct_change()['Count_3'].values
        if len(values_mnn) != 10:
            break

        deficit = ((region_covid - values_mnn) > 1).astype(int)[2:]
        proficit = ((values_mnn - region_covid) > 1).astype(int)[2:]

        mnn_values = np.array([mnn] * len(months))
        cur = np.vstack([mnn_values, region_values, months, deficit, proficit]).T
        train_mtx = np.vstack([train_mtx, cur])
        
#         if i > 1:
#             break
#     break

CPU times: user 11.3 s, sys: 979 ms, total: 12.3 s
Wall time: 12.3 s


In [154]:
region

'Чукотский автономный округ'

In [153]:
region in covid['name_store'].unique()

True

In [155]:
train = pd.DataFrame(train_mtx, columns=['mnn', 'region', 'month', 'deficit', 'proficit'])
train

,mnn,region,month,deficit,proficit
0,МНН №269,Владимирская область,3,0,0
1,МНН №269,Владимирская область,4,0,0
2,МНН №269,Владимирская область,5,1,0
3,МНН №269,Владимирская область,6,1,0
4,МНН №269,Владимирская область,7,0,0
...,...,...,...,...,...
7531,МНН №865,Чеченская республика,6,0,0
7532,МНН №865,Чеченская республика,7,1,0
7533,МНН №865,Чеченская республика,8,1,0
7534,МНН №865,Чеченская республика,9,0,0


In [165]:
names = pd.read_csv('spb/names_all.csv')
names

,name_store,lat_lon_store,name_sales,lat_lon_sales,name_covid,lat_lon_name_covid
0,Алтайский край,"(52.129671, 82.530022)",Алтайский край,"(52.129671, 82.530022)",Алтайский край,"(52.129671, 82.530022)"
1,Амурская область,"(53.413346, 127.728073)",Амурская область,"(53.413346, 127.728073)",Амурская обл.,"(53.413346, 127.728073)"
2,Архангельская область,"(63.637521, 43.33667)",Архангельская область,"(63.637521, 43.33667)",Архангельская обл.,"(63.637521, 43.33667)"
3,Астраханская область,"(46.851463, 47.466198)",Астраханская область,"(46.851463, 47.466198)",Астраханская обл.,"(46.851463, 47.466198)"
4,Белгородская область,"(50.872237, 37.303207)",Белгородская область,"(50.872237, 37.303207)",Белгородская обл.,"(50.872237, 37.303207)"
...,...,...,...,...,...,...
70,Хабаровский край,"(51.695886, 136.637043)",Хабаровский край,"(51.695886, 136.637043)",Хабаровский край,"(51.695886, 136.637043)"
71,Челябинская область,"(54.446204, 60.39565)",Челябинская область,"(54.446204, 60.39565)",Челябинская обл.,"(54.446204, 60.39565)"
72,Чукотский автономный округ,"(65.982613, 174.43232)",Чукотский автономный округ,"(65.982613, 174.43232)",Чукотский АО,"(65.982613, 174.43232)"
73,Ямало-Ненецкий автономный округ,"(67.808603, 75.199571)",Ямало-Ненецкий автономный округ,"(67.808603, 75.199571)",Ямало-Ненецкий АО,"(67.808603, 75.199571)"


In [171]:
len(set(train['region']) & set(fabric['Регион']))

31

In [168]:
len(set(names['name_sales']) & set(fabric['Регион']))

48

In [170]:
len(set(names['name_sales']) & set(fabric['Регион']))

30

In [163]:
fabric = pd.read_csv('spb/fabric.csv')
fabric.pop('Unnamed: 0')
fabric

,Регион,Кол.Завод
0,Алтайский край,4
1,Амурская область,1
2,Архангельская область,1
3,Владимирская область,2
4,Воронежская область,1
5,Ивановская область,1
6,Иркутская область,1
7,Кемеровская область,2
8,Кировская область,1
9,Краснодарская край,2


In [156]:
train['mnn'].nunique(), train['region'].nunique()

(20, 83)

In [158]:
train.to_csv('spb/train_v1.csv', index=False)

In [112]:
df_mnn

,name,Ковид,Count_3,month_,prod_norm_name
7777,Удмуртская республика,1,3584,8,МНН №806
25953,Удмуртская республика,1,26,8,МНН №806
50208,Удмуртская республика,1,1,8,МНН №806
75140,Удмуртская республика,1,29,8,МНН №806
95682,Удмуртская республика,1,20,8,МНН №806
...,...,...,...,...,...
4156493,Удмуртская республика,1,36,1,МНН №806
4175923,Удмуртская республика,1,287,1,МНН №806
4186107,Удмуртская республика,1,2618,1,МНН №806
4211516,Удмуртская республика,1,12,1,МНН №806


In [108]:
values_mnn

month_
9    NaN
10   NaN
Name: Count_3, dtype: float64

In [105]:
df['prod_norm_name'].unique()

array(['МНН №269', 'МНН №382', 'МНН №2136', 'МНН №266', 'МНН №806',
       'МНН №1132', 'МНН №1532', 'МНН №1207', 'МНН №138', 'МНН №720',
       'МНН №360', 'МНН №178', 'МНН №4', 'МНН №2200', 'МНН №1335',
       'МНН №1795', 'МНН №1774', 'МНН №865', 'МНН №166', 'МНН №1725',
       'МНН №942', 'МНН №1726'], dtype=object)

In [104]:
df_mnn[['month_', 'Count_3']].groupby('month_') \
                                     .sum()

,Count_3
month_,
3,36
4,30
5,59
6,55
7,34
8,36
9,34
10,37


In [101]:
covid.query('name_store == @region') \
                        .resample('MS')['Заражений за день'] \
                        .sum().pct_change().shift().values

array([        nan,         nan, -0.20360982, -0.34810288, -0.23359375,
       -0.18654434,  0.65288221,  1.6732373 , -0.15087918, -0.16315965])

In [103]:
region_covid.shape, values_mnn.shape

((10,), (8,))

In [95]:
df_mnn = df.query('prod_norm_name == @mnn')
values_mnn = df_mnn[['month_', 'Count_3']].groupby('month_') \
                             .sum().diff().pct_change()['Count_3'].values

deficit = ((region_covid - values_mnn) > 1).astype(int)[2:]
proficit = ((values_mnn - region_covid) > 1).astype(int)[2:]
mnn_values = np.array([mnn] * len(months))


ValueError: operands could not be broadcast together with shapes (10,) (8,) 

In [97]:
region_covid.shape

(10,)

In [93]:
np.vstack([mnn_values, region_values, months, deficit, proficit]).T

array([['МНН №1795', 'Владимирская область', '3', '0', '1'],
       ['МНН №1795', 'Владимирская область', '4', '0', '0'],
       ['МНН №1795', 'Владимирская область', '5', '1', '0'],
       ['МНН №1795', 'Владимирская область', '6', '1', '0'],
       ['МНН №1795', 'Владимирская область', '7', '0', '1'],
       ['МНН №1795', 'Владимирская область', '8', '1', '0'],
       ['МНН №1795', 'Владимирская область', '9', '0', '0'],
       ['МНН №1795', 'Владимирская область', '10', '0', '0']],
      dtype='<U21')

In [92]:
values_mnn.shape, deficit.shape, proficit.shape, mnn_values.shape

((8,), (8,), (8,), (8,))

In [87]:
df_mnn

,name,Ковид,Count_3,month_,prod_norm_name
218046,Владимирская область,1,36,8,МНН №1774
663066,Владимирская область,1,30,4,МНН №1774
1093980,Владимирская область,1,34,7,МНН №1774
1549869,Владимирская область,1,55,6,МНН №1774
1984864,Владимирская область,1,59,5,МНН №1774
2386135,Владимирская область,1,36,3,МНН №1774
2820350,Владимирская область,1,37,10,МНН №1774
3325476,Владимирская область,1,34,9,МНН №1774


In [82]:
mnn_values.shape, values_mnn.shape, deficit.shape, proficit.shape, months.shape

((8,), (8,), (8,), (8,), (8,))

In [68]:
train_mtx

array([['МНН №269', 'Владимирская область', '3', '0', '0'],
       ['МНН №269', 'Владимирская область', '4', '0', '0'],
       ['МНН №269', 'Владимирская область', '5', '1', '0'],
       ['МНН №269', 'Владимирская область', '6', '1', '0'],
       ['МНН №269', 'Владимирская область', '7', '0', '0'],
       ['МНН №269', 'Владимирская область', '8', '1', '0'],
       ['МНН №269', 'Владимирская область', '9', '1', '0'],
       ['МНН №269', 'Владимирская область', '10', '0', '0'],
       ['МНН №382', 'Владимирская область', '3', '0', '1'],
       ['МНН №382', 'Владимирская область', '4', '0', '0'],
       ['МНН №382', 'Владимирская область', '5', '0', '0'],
       ['МНН №382', 'Владимирская область', '6', '0', '0'],
       ['МНН №382', 'Владимирская область', '7', '0', '0'],
       ['МНН №382', 'Владимирская область', '8', '1', '0'],
       ['МНН №382', 'Владимирская область', '9', '0', '0'],
       ['МНН №382', 'Владимирская область', '10', '1', '0'],
       ['МНН №2136', 'Владимирская обл

In [62]:
region_values

array(['Владимирская область', 'Владимирская область',
       'Владимирская область', 'Владимирская область',
       'Владимирская область', 'Владимирская область',
       'Владимирская область', 'Владимирская область'], dtype='<U20')

In [32]:
months.dtype

dtype('int64')

In [36]:
pd.DataFrame(train_mtx)

,0,1,2,3,4
0,МНН №269,Владимирская область,3,0,0
1,МНН №269,Владимирская область,4,0,0
2,МНН №269,Владимирская область,5,1,0
3,МНН №269,Владимирская область,6,1,0
4,МНН №269,Владимирская область,7,0,0
5,МНН №269,Владимирская область,8,1,0
6,МНН №269,Владимирская область,9,1,0
7,МНН №269,Владимирская область,10,0,0
8,МНН №382,Владимирская область,3,0,1
9,МНН №382,Владимирская область,4,0,0
